In [2]:
import pandas as pd
import numpy as np
import os, re, datetime


In [47]:
df = pd.read_csv("outOfJob.csv")

In [48]:
origin = '/pylon5/sy5fp1p/ehanna/logs/hosts/'

In [49]:
pattern = re.compile('(\d+-\d+-\d+)T(\d+:\d+:\d+)-(\d+:\d+)\s(.*?)\.pvt.*(-\s)+(.*)')

In [50]:
df = df.set_index('Unnamed: 0')
df.index.name = 'Index'

In [55]:
def outOfJobErrorText(df):
    corpus = []
    count = 0
    for i in range(len(df)):
        print(i)
        for column in df.columns:
            if df.loc[df.index[i],column]>0:
                date = df.index[i].split('_')[0]
                timeString = df.index[i].split('_')[1]
                time = datetime.datetime.strptime(timeString,'%H:%M:%S')
                timeToStartSeconds = (int(time.hour)*3600+int(time.minute)*60+int(time.second))-(1*60)
    #             print(timeToStartSeconds)
                timeToStart = str(datetime.timedelta(seconds=timeToStartSeconds))
                fileName = origin +column+'.pvt.bridges.psc.edu/'+date+'-'+column+'.pvt.bridges.psc.edu.log'
                with open(fileName) as file:
                    fileCorpus = ''
                    for line in file:
                        if 'not responding' not in line and 'nfs: server' not in line:
                            match = re.search(pattern,line)
                            if match is not None:
                                time = match.group(2)
                                if time>timeToStart:
                                    text = match.group(6)
                                    fileCorpus+=text+'\n'
                                elif time>timeString:
                                    break
                    corpus.append(fileCorpus)
    return corpus

In [56]:
corpus = outOfJobErrorText(df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166


[' DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n DHCPREQUEST on eno1 to 10.4.200.55 port 67 (xid=0x7b94874a)\n [f62a09] <passwd="monitor"> failed to bind to LDAP server ldaps://br004.pvt.bridges.psc.edu: Timed out: Resource temporarily unav

In [59]:
allText = ''
for i in corpus:
    if len(i)>0:
        allText+=i+'\n'
file = open("OutOfJobPredecessors.txt","w")
file.write(allText)
file.close()

In [4]:
textUniquePattern = re.compile('[a-zA-Z0-9\.\:]*[\d]+[a-zA-Z0-9\.\:]*')
# testPattern = re.compile('\d+.\d+.\d+.\d+')

In [5]:
uniqueTexts = []
textFrequencies = []
with open("OutOfJobPredecessors.txt") as file:
    for line in file:
        line = line.strip('\n')
        line = re.sub(textUniquePattern,"",line)
        if line not in uniqueTexts:
            uniqueTexts.append(line)
            textFrequencies.append(1)
        else:
            textFrequencies[uniqueTexts.index(line)]+=1

In [9]:
uniqueTexts

[' DHCPREQUEST on  to  port  (xid=)',
 ' Did not receive identification string from ',
 ' userauth_hostbased mismatch: client sends , but we resolve  to ',
 ' Postponed publickey for monitor from  port   RSA , client user "root", client host "" [preauth]',
 ' Created slice user-',
 ' Starting user-',
 ' Started Session  of user root.',
 ' Starting Session  of user root.',
 ' (root) CMD (/usr/sbin/naemon_update.sh)',
 ' (root) INFO (Job execution of per-minute job scheduled for  delayed into subsequent minute  Skipping job run.)',
 ' Accepted publickey for monitor from  port   RSA ',
 ' New session  of user monitor.',
 ' Started Session  of user monitor.',
 ' Starting Session  of user monitor.',
 ' pam_unix(sshd:session): session opened for user monitor by (uid=)',
 ' bash load psc_path slurm intel  [monitor().pscstaff()] [monitor().pscstaff()] = TCL_OK()',
 ' Received disconnect from   disconnected by user',
 ' pam_unix(sshd:session): session closed for user monitor',
 ' Removed sessio

In [10]:
allTexts = []
for i in range(len(uniqueTexts)):
    allTexts.append([uniqueTexts[i],textFrequencies[i]])

In [15]:
 allTexts = sorted(allTexts,key=lambda x: x[1],reverse=True)

In [33]:
len(uniqueTexts)

2108

In [17]:
text = ''
for i in range(len(allTexts)):
    text+= allTexts[i][0]+' '+str(allTexts[i][1])+'\n'
file = open("OutOfJobuniquePred.txt","w")
file.write(text)
file.close()

In [18]:
text

' DHCPREQUEST on  to  port  (xid=) 818985\n userauth_hostbased mismatch: client sends , but we resolve  to  61708\n [] <group="hduser"> failed to bind to LDAP server ldaps:// Can\'t contact LDAP server: Connection timed out 54264\n Started Session  of user root. 38992\n Starting Session  of user root. 38992\n pam_access(crond:account): access denied for user `pcp\' from `cron\' 36874\n   _ send_idle_message: sending idle message  25015\n   _ set_link_state: current ACTIVE, new ACTIVE  24960\n Removed slice user- 22806\n Stopping user- 22806\n Created slice user- 22805\n Starting user- 22805\n Did not receive identification string from  21276\n [] <group/member="pcp"> no available LDAP server found: Server is unavailable: Connection timed out 20844\n Postponed publickey for monitor from  port   RSA , client user "root", client host "" [preauth] 20506\n [] <group="hduser"> no available LDAP server found: Can\'t contact LDAP server: Connection timed out 19658\n Removed session  19529\n Re